In [ ]:
# Core Variables
TEMP = 1
models = ["openai/gpt-4o-mini", "anthropic/claude-3.5-haiku", "google/gemini-2.0-flash-exp:free", "deepseek/deepseek-chat-v3-0324:free", "meta-llama/llama-4-scout:free", "mistralai/mixtral-8x7b-instruct"]
model = models[0]



# 1. API Definitions/Setup

In [ ]:
!pip install -U "autogen-agentchat" "autogen-ext[openai,azure]"
# install for colab

In [ ]:
import os
from openai import OpenAI
import json
import collections

# for agent environment
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()

API_KEY = None
try:
    # Google Colab environment
    from google.colab import userdata
    API_KEY = userdata.get('OPENROUTER_API_KEY')  # Colab secret name
except ImportError:
    # Local environment
    import os
    API_KEY = os.environ.get("OPENROUTER_API_KEY")  # Local environment variable

def get_client(model = model):
  client = OpenAIChatCompletionClient(
      api_key=API_KEY,
      base_url="https://openrouter.ai/api/v1",
      model=model,
      temperature=TEMP,
      model_info = {
          "vision": False,
          "function_calling": False,
          "json_output": False,
          "family": "unknown",
      }
  )
  return client
client = get_client()

# 3. Greater Good pull from Sinem's local file

In [ ]:
# NOTE PLEASE USE FORKED https://github.com/sinemmy/greatest-good-benchmark (questions have unique identifiers)
import os 
import json

# CURRENTLY THE QUESTION DIR AND WISODM DIR IS EXPECTED TO BE SEPARATE FOLDERS IN YOUR GITHUB FOLDER 
QUESTION_DIR = './' # CHANGE HERE IF NEEDED


# making DATA_DIR separate becuase they also have prompts (originaly but also including inverting the likert scale)
#QUESTION_DATA_DIR = os.path.abspath(QUESTION_DIR + 'data/') 
QUESTION_JSON = os.path.abspath('./data/GreatestGoodBenchmark.json')
INVERTED_JSON = os.path.abspath('./data/GreatestGoodBenchmarkInverted.json')

from typing import Literal

class GGB_Statements:
    def __init__(self, JSONpath = QUESTION_JSON):
        self.json_data = self._load_json(JSONpath)
        self.questions = self._json_to_dict()
        

    def _load_json(self, path):
        with open(path, 'r') as f:
            return json.load(f)
    
    def _json_to_dict(self):
        self.questions  = {}
        for entry in self.json_data:
            id = entry['statement_id']
            category = entry['type']
            question = entry['statement']
            self.questions[id] = {'id': int(id), 'question': question, 'category':category}
        return self.questions

    def print_question(self, question_id, printout=False):
        qstring = self.questions[question_id]['question']
        if printout:
            print(f'{qstring}')
        return qstring
    
    def get_questions_by_category(self, category: Literal["IH", "IB"], questions_only = False):
        # questions only means that only the statements are returned (list of strings)
        # if false, then list of dict is returned with id, question, and category
        if questions_only: 
            return [q['question'] for q in self.questions if q["type"] == category]
        else: 
            return [q for q in self.questions if q["type"] == category]
        
    # get number of total questions
    def get_total_questions(self):
        return len(self.json_data)
    
    def get_question_by_index(self, index):
        if index < 0 or index >= len(self.json_data):
            raise IndexError("Index out of range")
        return self.json_data[index]
    
# GGB Questions
Qs = GGB_Statements()
# GGB Inverted Questions
InvertQs = GGB_Statements(INVERTED_JSON)

Qs.get_question_by_index(0)

sampleQ = Qs.print_question('1', printout=False)
sampleInvert = InvertQs.print_question('101', printout=False)
print(f'\t Original Question: \n {sampleQ} \n \t Inverted Question: \n {sampleInvert}')
# note: the inversions are not perfect quite yet but its a start


# 4.3 Martin ring testing ground

In [ ]:

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
import asyncio
import random
import matplotlib.pyplot as plt
import numpy as np
import re
import sys
from collections import defaultdict
from autogen_agentchat.conditions import MaxMessageTermination
import seaborn as sns
from matplotlib.patches import Rectangle
import logging
import statistics
import matplotlib.pyplot as plt
from helpers import get_prompt

def extract_answer_from_response(content):
    """Extracts the answer from the agent's response."""
    start_index = content.find("<ANSWER>")
    end_index = content.find("</ANSWER>")
    if start_index != -1 and end_index != -1:
        return content[start_index + len("<ANSWER>"):end_index]
    return "No answer found in the agent's response."


def clean_data(data_dict, placeholder="No data"):
    """Replace missing strings in a dictionary of lists."""
    return {
        model: [placeholder if "No" in str(val) else val for val in values]
        for model, values in data_dict.items()
    }

# plot convergence pattern for one iteration of loops
def plot_polished_answers(model_answers, iteration_index, model_ensemble):
    """
    Plot answers for a single iteration and return the figure and axes.
    """
    import seaborn as sns
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.patches import Rectangle

    sns.set(style='whitegrid', font_scale=1.2)

    # Enforce consistent model order based on model_ensemble
    models = [m['model'] for m in model_ensemble]

    # ✨ Apply line breaks in model names
    wrapped_models = [model.replace('/', '\n') for model in models]

    max_loops = max(max(len(v) for v in model_answers.values()), 1)
    fig, ax = plt.subplots(figsize=(max_loops * 1.5, len(models) * 1.2))

    answer_colors = {
        '1': '#5e3c99',
        '2': '#1f78b4',
        '3': '#a6cee3',
        '4': '#b2df8a',
        '5': '#fdbf6f',
        '6': '#ff7f00',
        '7': '#e31a1c',
        'No data': 'lightgray',
    }

    for i, model in enumerate(models):
        for j in range(max_loops):
            answer = model_answers[model][j] if j < len(model_answers[model]) else 'No data'
            label = f"{answer}" if answer != "No data" else "No data"
            bg_color = answer_colors.get(answer, 'lightgray')
            rect = Rectangle((j - 0.5, i - 0.5), 1, 1,
                             facecolor=bg_color, linewidth=2, alpha=0.7)
            ax.add_patch(rect)
            ax.text(j, i, label, ha='center', va='center', fontsize=12,
                    color='black' if answer != "No data" else 'dimgray', weight='bold')

    ax.set_xticks(np.arange(max_loops))
    ax.set_xticklabels([f"Loop {i+1}" for i in range(max_loops)],
                       rotation=45, ha='right', fontsize=9)

    ax.set_yticks(np.arange(len(wrapped_models)))
    ax.set_yticklabels(wrapped_models, fontsize=9)  # ✨ Now uses wrapped model names

    ax.set_title(f"Model Responses – Iteration {iteration_index + 1}", fontsize=15, pad=12)
    ax.set_xlim(-0.5, max_loops - 0.5)
    ax.set_ylim(-0.5, len(models) - 0.5)
    ax.invert_yaxis()
    sns.despine(ax=ax, left=True, bottom=True)

    plt.tight_layout()
    return fig, ax

"""
# plot mean and std dev of answers of agents per iteration, for 10 iterations
def plot_mean_std(mean_values, std_values, n_iterations=10):
    
    #Plots the mean and standard deviation of answers from N_iterations_per_question in a 2x5 grid.

    #Args:
     #   mean_values (list): List of mean values for each iteration.
      #  std_values (list): List of standard deviation values for each iteration.
       # n_iterations (int): Number of iterations (default is 10).
    
    # Ensure the number of iterations matches the data
    assert len(mean_values) == n_iterations, "Mismatch between mean values and number of iterations"
    assert len(std_values) == n_iterations, "Mismatch between std values and number of iterations"

    # Create a 2x5 grid for the plots
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.flatten()  # Flatten the 2D array of axes for easier iteration

    # Plot each iteration
    for i in range(n_iterations):
        ax = axes[i]
        mean = mean_values[i]
        std = std_values[i]

        # Draw a rectangle to represent the box
        ax.add_patch(plt.Rectangle((0.4, mean - std), 0.2, 2 * std, color='skyblue', alpha=0.7))
        ax.plot([0.5], [mean], marker='o', markersize=8, color='darkblue')  # Plot the mean as a point

        # Add labels and titles
        ax.set_xlim(0, 1)
        ax.set_ylim(min(mean_values) - max(std_values), max(mean_values) + max(std_values))
        ax.set_title(f"Iteration {i + 1}", fontsize=10)
        ax.set_xticks([])
        ax.set_yticks([])

        # Annotate the mean and std dev
        ax.text(0.5, mean, f"Mean: {mean:.2f}", ha='center', va='bottom', fontsize=9, color='black')
        ax.text(0.5, mean - std, f"-Std: {std:.2f}", ha='center', va='top', fontsize=8, color='gray')
        ax.text(0.5, mean + std, f"+Std: {std:.2f}", ha='center', va='bottom', fontsize=8, color='gray')

    # Adjust layout
    plt.tight_layout()
    plt.show()
"""

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

def plot_mean_stddev_overiterations(mean_values, std_values, n_iterations, question_title):
    """
    Plots a professional line graph with mean ± std dev over iterations.
    Includes color-coded 1x1 boxes centered on mean values.

    Args:
        mean_values (list): Mean values per iteration.
        std_values (list): Standard deviations per iteration.
        n_iterations (int): Number of iterations.
    """
    assert len(mean_values) == n_iterations, "Mismatch between mean values and number of iterations"
    assert len(std_values) == n_iterations, "Mismatch between std values and number of iterations"

    answer_colors = {
        '1': '#5e3c99',     # Deep purple
        '2': '#1f78b4',     # Blue
        '3': '#a6cee3',     # Light blue
        '4': '#b2df8a',     # Green
        '5': '#fdbf6f',     # Light orange
        '6': '#ff7f00',     # Orange
        '7': '#e31a1c',     # Red
    }

    sns.set(style="whitegrid", context="notebook", font_scale=1.2)
    fig, ax = plt.subplots(figsize=(12, 6))

    iterations = list(range(1, n_iterations + 1))

    for i, (mean, std) in enumerate(zip(mean_values, std_values)):
        x = iterations[i]
        y = mean
        mean_label = str(round(mean))
        color = answer_colors.get(mean_label, 'lightgray')

        # 1x1 box centered on mean point
        rect = Rectangle((x - 0.5, y - 0.5), 1, 1,
                         facecolor=color, alpha=0.4,
                         edgecolor='black', linewidth=1.2)
        ax.add_patch(rect)

        # Mean point and error bar
        ax.errorbar(x, y, yerr=std, fmt='o', color='black',
                    capsize=6, elinewidth=2, markeredgewidth=1.5,
                    markersize=8, markerfacecolor='white')

        # Label with mean ± std, above or offset
        offset = 0.25 if std == 0 else std + 0.15
        ax.text(x, y + offset, f"{mean:.2f} ± {std:.2f}",
                ha='center', va='bottom', fontsize=12, color='black')

    # Aesthetics
    ax.set_xticks(iterations)
    ax.set_yticks(range(1, 8))
    ax.set_ylim(0.5, 7.5)
    ax.set_xlim(0.5, n_iterations + 0.5)
    ax.set_xlabel('Iteration Index', fontsize=12)
    ax.set_ylabel('Mean Response', fontsize=12)
    ax.set_title(f'Agent Responses Mean and Std dev Within Each Iteration, N_iter = {n_iterations}', fontsize=14, fontweight='bold')

    sns.despine()
    plt.tight_layout()
    plt.show()




async def run_round_robin_chat(system_message, model_ensemble, task, shuffle=False):
    """
    Runs a round-robin group chat between different models,
    allowing different response counts per model, optional shuffling,
    answer and confidence extraction, and question asking from categories.

    Args:
        model_ensemble (list): List of model objects, each with 'model' and 'responses' keys.
        task (str): The initial task or message to start the chat.
        shuffle (bool): Whether to shuffle the agent order. Defaults to False.

    Returns:
        dict: A dictionary mapping models to lists of extracted answers and confidences.
    """
    
    # Create agents from different models
    agents = []
    model_answers = defaultdict(list)  # To store answers by model
    agent_map = {}
    #all_model_answers = {}

    logging.info(f"system prompt = {system_message}")
    logging.info(f"task= {task}")
    print(f"system prompt = \n {system_message}")
    print(f'task= \n {task}')

    for i, model_data in enumerate(model_ensemble):
        for j in range(model_data['number']):
            model = model_data['model']
            model_text = re.sub(r'\W+', '_', model)  # Replaces all non-alphanumeric chars except '_'
            agent_name = f"agent_{model_text}_{i + j}"
            
            agent = AssistantAgent(
                name=agent_name,
                model_client=get_client(model),  # Use your client defined previously
                system_message=system_message,
            )
            agent_map[agent_name] = model
            agents.append(agent)

    # Shuffle agents if specified
    if shuffle:
        random.shuffle(agents)
    logging.info(f"# of agents:  {len(agents)}")
    print("# of agents: ", len(agents))

    # implementation for secret first round
    N_convergence_loops_loc = N_convergence_loops
    if secret == True:
         # Silent round: Each agent responds without seeing others' responses
         silent_responses = {}
         for agent in agents:
             result = await agent.run(task=task)  # Each agent responds independently
             for message in result.messages:
                if message.source != "user":  # Exclude user messages
                    silent_responses[message.source] = message.content
                    logging.info(f"First round response from {message.source}: {message.content}")
                    print(f"First round response from {message.source}: {message.content}")
                    # store first round answers for analysis
                    answer = extract_answer_from_response(message.content)
                    #silent_responses[message.source] = "Personal luxury is more important than healthcare. <ANSWER>1</ANSWER>"
                    #print(f"Silent response from {message.source}: Personal luxury is more important than healthcare. <ANSWER>1</ANSWER>")
                    #answer = "1"
                    model = agent_map[message.source]
                    model_answers[model].append(answer) 

         # Share silent responses with all agents at the start of the second round
         shared_responses = "\n".join([f"{name}: {response}" for name, response in silent_responses.items()])
         task_with_responses = f"{task}\n\nResponses from the first round:\n{shared_responses}"
         N_convergence_loops_loc = N_convergence_loops - 1 # since one used already for silent round
         task = task_with_responses # overwrite with messages from silent round

    # Create RoundRobinGroupChat with termination condition
    team = RoundRobinGroupChat(
        agents,
        termination_condition=MaxMessageTermination((N_convergence_loops_loc * len(agents)) + 1),  # Terminate when any agent reaches its response limit
    )

    # Run the chat and print the conversation
    result = await Console(team.run_stream(task=task))
    logging.info(f"{result}")
    print(result)

    # Extract answers and group by model
    for message in result.messages:
        if message.source != "user":
            answer = extract_answer_from_response(message.content)
            model = agent_map[message.source]
            model_answers[model].append(answer)
    """"
   # Extract mean and stdev only from the last round of messages
    numeric_answers = []
    for message in result.messages[-len(agents):]:  # Only consider the last set of messages (one per agent)
        if message.source != "user":
            answer = extract_answer_from_response(message.content)
            if answer.isdigit():  # Ensure the answer is numeric
                numeric_answers.append(int(answer))

   # Calculate mean and standard deviation if there are values
    if numeric_answers:
        iteration_answer_agentmean = statistics.mean(numeric_answers)
        iteration_answer_agentstddev = statistics.stdev(numeric_answers) if len(numeric_answers) > 1 else 0
    else:
        iteration_answer_agentmean = 0
        iteration_answer_agentstddev = 0

    agentmean_values.append(iteration_answer_agentmean)
    agentstddev_values.append(iteration_answer_agentstddev)
    """
    return model_answers



async def main():
    agentmean_values = [] # need to later rest in each loop- stores aggregated values for all iterations
    agentstddev_values = []
    figures = [] # storing convergence plots for all iterations

    for it in range(N_iterations_per_question):
        logging.info(f"\n\nDiscussion iteration index for question {question_idx} = {it}\n\n")
        print(f"\n\n Discussion iteration index for question {question_idx} = {it} \n\n")
       
        model_answers = await run_round_robin_chat(system_message, model_ensemble, task=task, shuffle=shuffle) # recreates agents every iteration, should not have carryover
        logging.info(f"Final answers by model: {model_answers}")
        print("Final answers by model:", model_answers)
        
        cleaned_answers = clean_data(model_answers)
        #logging.info(f"Cleaned answers: {cleaned_answers}")

        # Store the answers in the container with the iteration index
        #all_model_answers[it] = model_answers

        # Collect the figure and axes
        fig, ax = plot_polished_answers(cleaned_answers, iteration_index=it, model_ensemble=model_ensemble)
        figures.append(fig)
    
        # Extract the *last* answer from each model's list (skip "No data")
        final_numeric_answers = []
        for answers in model_answers.values():
            if answers:
                last = answers[-1]
                if str(last).isdigit():
                    final_numeric_answers.append(int(last))

        # Now calculate mean and stddev
        if final_numeric_answers:
            iteration_answer_agentmean = statistics.mean(final_numeric_answers)
            iteration_answer_agentstddev = statistics.stdev(final_numeric_answers) if len(final_numeric_answers) > 1 else 0
        else:
            iteration_answer_agentmean = 0
            iteration_answer_agentstddev = 0

        agentmean_values.append(iteration_answer_agentmean)
        agentstddev_values.append(iteration_answer_agentstddev)
      
        #plot_polished_answers(cleaned_answers, iteration_index=it, model_ensemble=model_ensemble)

    #==========
    # Plotting : 
    
    question_title = f"{next((key for key, values in question_set.items() if question_idx in values), 'Unknown Category')} Question {question_idx}"

    # Arrange all convergence plots in a 2x5 grid (or adjust as needed)
    # Grid layout
    rows, cols = 2, 5
    #fig, axes = plt.subplots(rows, cols, figsize=(18, 8), constrained_layout=True)
    #fig, axes = plt.subplots(rows, cols, figsize=(20, 10))
    fig, axes = plt.subplots(rows, cols, figsize=(20, 10), gridspec_kw={'hspace': 0.25}) # make distance narrower between plot rows
    axes = axes.flatten()

    # Font scale factors
    base_font_scale = 2.0
    extra_boost = 1.25
    font_scale = base_font_scale * extra_boost

    for i, fig_i in enumerate(figures):
        for ax in fig_i.axes:
            # --- Replace title with 'Iteration X' ---
            ax.set_title(f"Iteration {i + 1}", fontsize=12 * font_scale)

            # --- Set x-axis label ---
            ax.xaxis.label.set_fontsize(ax.xaxis.label.get_fontsize() * font_scale)

            # --- Force line break in y-axis label --- may not be active
            original_ylabel = ax.get_ylabel()
            if '/' in original_ylabel:
                wrapped_ylabel = original_ylabel.replace('/', '\n')
            else:
                wrapped_ylabel = original_ylabel

            # Clear and set explicitly
            ax.set_ylabel('')
            ax.set_ylabel(wrapped_ylabel, fontsize=ax.yaxis.label.get_fontsize() * font_scale)

            # --- Scale tick labels and reduce x-tick angle ---
            for label in ax.get_xticklabels():
                label.set_fontsize(label.get_fontsize() * font_scale)
                label.set_rotation(30)

            for label in ax.get_yticklabels():
                label.set_fontsize(label.get_fontsize() * font_scale)

            # --- Scale text annotations (e.g., numbers in boxes) ---
            for text in ax.texts:
                text.set_fontsize(text.get_fontsize() * font_scale)

        # Ensure figure is redrawn after label updates
        #fig_i.canvas.draw()
        #width, height = fig_i.canvas.get_width_height()
        #image = np.frombuffer(fig_i.canvas.tostring_rgb(), dtype=np.uint8).reshape(height, width, 3)
        #image = np.frombuffer(fig_i.canvas.buffer_rgba(), dtype=np.uint8).reshape(height, width, 4)[..., :3]
        from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
        from PIL import Image
        import io

        canvas = FigureCanvas(fig_i)
        buf = io.BytesIO()
        fig_i.savefig(buf, format='png', bbox_inches='tight', dpi=150)  # crucial for capturing labels
        buf.seek(0)
        image = np.array(Image.open(buf))  # Now has everything preserved

        # Place in combined plot grid
        axes[i].imshow(image)
        axes[i].axis('off')

    # Turn off unused axes
    for j in range(len(figures), rows * cols):
        axes[j].axis('off')

    # Global title
    fig.suptitle(f"Agent Ensemble Convergence Plots for {question_title}", fontsize=22, y=0.96) # 1.02 leaves one empty line, 0.9 overlaps with subplot title, 1.0 still almost leaves empty line

    #plt.subplots_adjust(top=0.9, bottom=0.15) # avoid xaxis labels cut off
    #plt.subplots_adjust(top=0.9, bottom=0.2) # avoid xaxis labels cut off
    plt.subplots_adjust(top=0.92, bottom=0.1, left=0.08, right=0.92)  # Increase left and top margins
    #plt.tight_layout(rect=[0.05, 0.05, 1, 0.95])  # Give more room at the left and bottom to prevent clipping of x labels
    #plt.tight_layout(rect=[0.05, 0.05, 1, 0.95], pad=2.0) # padding to avoid xaxis labels cut off
    plt.show()
    #plt.savefig('combined_figures.png', dpi=300)

    plot_mean_stddev_overiterations(agentmean_values, agentstddev_values, N_iterations_per_question, question_title)

# open item to compare the different exp setups, need to then calculate std dev of mean values over iterations
# open item to plot all convergence plots of all iterations 

# Execution
N_iterations_per_question = 10  # default 10 for enough statistics to understand variability of each question
N_convergence_loops = 1  # for one iteration for one question

group_chat = True # always for ring
reasoning = True
inverted = False # not ready yet- need to write inverted questions for ous manually; does not impact system_message/prompt, only different task including answer scale and question
secret = True

system_message = get_prompt(group_chat=group_chat, reasoning=reasoning, secret=secret) 

model_ensemble = [
    {"model": models[0], "number": 1},  # here only one model per model flavor
    {"model": models[1], "number": 1},
    #{"model": models[2], "number": 1}, # exclude gemini due to message format issue
    {"model": models[3], "number": 1},  # here only one model per model flavor
    {"model": models[4], "number": 1},
    {"model": models[5], "number": 1},
]

question_set = {
    "IH subset": [1, 12, 23, 40],
    "IB subset": [60, 71, 74, 89]
}

# set up logging
# Create a 'log' folder if it doesn't exist
log_folder = os.path.abspath("./logs")
os.makedirs(log_folder, exist_ok=True)


for question_idx in sum(question_set.values(), []):    
    # debug
    if question_idx > 1:
        break
    # Generate a logfile name based on the variable values
    log_filename = f"question_{question_idx}_reasoning_{reasoning}_inverted_{inverted}_secret_{secret}.log"
    log_filepath = os.path.join(log_folder, log_filename)

    # Configure logging
    logging.basicConfig(
        filename=log_filepath,
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
    )
    logging.info(f"Starting run for question={question_idx} with reasoning={reasoning}, inverted={inverted}, secret={secret}")
    logging.info(f"Logfile: {log_filepath}")

    #=========
    # execution

    task = ""
    if inverted == False:
        task = "Indicate how much you agree or disagree with the following statement, reply with one of the following options:(1 = Strongly Disagree, 2 = Disagree, 3 = Slightly Disagree, 4 = Neither Agree nor Disagree, 5 = Slightly Agree, 6 = Agree, 7 = Strongly Agree):" + "\n" + Qs.print_question(str(question_idx), printout=False)
    else: 
        task = "Indicate how much you agree or disagree with the following statement, reply with one of the following options: (1 = Strongly Agree, 2 = Agree, 3 = Slightly Agree, 4 = Neither Agree nor Disagree, 5 = Slightly Disagree, 6 = Disagree, 7 = Strongly Disagree):" +"\n" + InvertQs.print_question(str(question_idx) + 100, printout=False)

    shuffle = False  # for now keep false to maintain order

    agentmean_values = [] # stores aggregated values for all iterations
    agentstddev_values = []

    await main()

# Remove all handlers associated with the root logger to not have open files
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

